A priori es probable que se sepa qué profesores harán cada clase para cada curso. Esto reduce considerablemente el tamaño del problema, por lo que tiene sentido armar un modelo reducido que considere esto.

In [466]:
import gurobipy as gp
from gurobipy import GRB
from operator import itemgetter
import itertools
import math
import numpy as np 
import json
from prettytable import PrettyTable


class MultiDimensionalArrayEncoder(json.JSONEncoder):
    def encode(self, obj):
        def hint_tuples(item):
            if isinstance(item, tuple):
                return {'__tuple__': True, 'items': item}
            if isinstance(item, list):
                return [hint_tuples(e) for e in item]
            if isinstance(item, dict):
                return {key: hint_tuples(value) for key, value in item.items()}
            else:
                return item

        return super(MultiDimensionalArrayEncoder, self).encode(hint_tuples(obj))
def hinted_tuple_hook(obj):
    if '__tuple__' in obj:
        return tuple(obj['items'])
    else:
        return obj
enc = MultiDimensionalArrayEncoder()

In [467]:
def formar_asignaciones(profesores_por_curso: dict):
    lista_asignaciones = []
    for curso in profesores_por_curso:
        for tupla in profesores_por_curso[curso]:
            lista_asignaciones.append((curso[:-1], curso[-1], tupla[0], tupla[1], ))
    return lista_asignaciones 

In [468]:
model = gp.Model("Generación de horarios de colegio")
# model.setParam('OutputFlag', 1)
f = open('parametros.json',encoding='utf8')

M = 10000
data = json.load(f, object_hook=hinted_tuple_hook)
dias = data['dias']
modulos = data['modulos']
ramos_con_modulos_seguidos = data['ramos_con_modulos_seguidos']
combinaciones = data['combinaciones']
horarios = data['horarios']
profesores_por_curso = data['profesores_por_curso']
ramos_por_curso = data['ramos_por_curso']
profesores = data['profesores']
asignaciones = formar_asignaciones(profesores_por_curso)

x = model.addVars(asignaciones,dias,modulos,vtype=GRB.BINARY, name="x")
s = model.addVars(asignaciones,dias,vtype=GRB.BINARY, name="s")

El profesor asignado hace la clase en algún momento

In [469]:
model.addConstrs((sum(x[curso[:-1],curso[-1],p,k,dia,modulo+1] for p,k in profesores_por_curso[curso]) == 1 \
    for curso in horarios for dia in horarios[curso] for modulo in range(horarios[curso][dia])),name="R1");

Cumplir con la cantidad de clases semanales

In [470]:
model.addConstrs((sum(x[curso[:-1],curso[-1],p,k,dia,j] for dia in dias for j in modulos) == ramos_por_curso[curso][k] \
    for curso in horarios for p,k in profesores_por_curso[curso]),name="R2");

No pasarse del módulo máximo (se asume que los horarios son continuos)

In [471]:
model.addConstrs((sum(x[curso[:-1],curso[-1],p,k,dia,j] for p,k in profesores_por_curso[curso]) == 0 \
    for curso in horarios for dia in horarios[curso] for j in modulos if j > horarios[curso][dia]),name="R3");

Los profesores solo pueden estar en un lugar al mismo tiempo

In [472]:
model.addConstrs((sum(x[i,c,p,k,dia,j] for i,c,_,k in asignaciones if _ == p) <= 1 \
    for p in profesores for dia in dias for j in modulos), name="R4");

Módulos de clases seguidos

In [473]:
model.addConstrs(sum(x[curso[:-1],curso[-1],p,k,dia,j] for j in modulos) <= 2 for dia in dias for curso in horarios for p,k in profesores_por_curso[curso]);

In [474]:
model.addConstrs(x[i,c,p,k,dia,j] + x[i,c,p,k,dia,z] <= 1 for i,c,p,k in asignaciones for dia in dias for j,z in list(set([(a+1,b+1) for a in range(horarios[i+c][dia]) for b in range(horarios[i+c][dia]) if a < b]) - set(combinaciones)));

In [475]:
model.addConstrs((M * s[i,c,p,k,d] >= sum(x[i,c,p,k,d,j] for j in modulos) for i,c,p,k in asignaciones for d in dias),name="RS1");

In [476]:
model.addConstrs((sum(s[curso[:-1],curso[-1],p,k,d] for d in dias) <= math.ceil(ramos_por_curso[curso][k] / 2) for curso in horarios for p,k in profesores_por_curso[curso] if k in ramos_con_modulos_seguidos),name="RS2");

In [477]:
model.setObjective(0, GRB.MINIMIZE)

model.write('model.lp')

# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)

model.update()
model.optimize()
model.write('out.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3698 rows, 2000 columns and 14840 nonzeros
Model fingerprint: 0xee41cc8f
Variable types: 0 continuous, 2000 integer (2000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 3140 rows and 1364 columns
Presolve time: 0.06s
Presolved: 558 rows, 636 columns, 3388 nonzeros
Variable types: 0 continuous, 636 integer (636 binary)

Root relaxation: objective 0.000000e+00, 378 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000  0.00%     -    0s

Explored 1 nodes (964 simplex iterations) in 0.09 seconds (0.06 work uni

In [478]:
def leer_output():
    lista_final = []
    with open('out.sol', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line[-2] == '1':
                line = line.strip('\n')
                lista_final.append(line)
    # print(lista_final)
    diccionario_dias = {
        'Lunes': 1,
        'Martes': 2,
        'Miércoles': 3,
        'Jueves': 4,
        'Viernes': 5
    }

    vuelta = {
        1: 'Lunes',
        2: 'Martes',
        3: 'Miércoles',
        4: 'Jueves',
        5: 'Viernes'
    }

    profesores_final = []
    for elem in lista_final:
        if elem[0] == "x":
            elem = elem[2:-3]
            elem = elem.split(',')
            profesores_final.append(elem)
    for i in range(len(profesores_final)):
        profesores_final[i][4] = diccionario_dias[profesores_final[i][4]]
    profesores_final = sorted(profesores_final, key= lambda x: x[5])
    profesores_final = sorted(profesores_final, key= lambda x: x[4])
    profesores_final = sorted(profesores_final, key= lambda x: x[1])

    for i in range(len(profesores_final)):
        profesores_final[i][4] = vuelta[profesores_final[i][4]]
    return profesores_final

def generar_horario(curso):
    nivel = curso[:-1]
    paralelo = curso[-1]

    clases_curso = []
    dias_semana = []
    ultimo_dia = None
    horario = PrettyTable(dias_semana)
    lista_auxiliar = []
    lista_dia = None
    for elem in leer_output():    
        if elem[0] == nivel and elem[1] == paralelo:
            clases_curso.append([elem[3], elem[4], elem[5], elem[2]])
            if elem[4] not in dias_semana:
                dias_semana.append(elem[4])
            if elem[4] != ultimo_dia:
                lista_auxiliar.append(lista_dia)
                lista_dia = []
                ultimo_dia = elem[4]
                lista_dia.append(ultimo_dia)
                elem[3] = elem[3].replace('_', ' ')
                elem[2] = elem[2].replace('_', ' ')
                lista_dia.append(f'{elem[3]} \n {elem[2]}')
            else:
                elem[3] = elem[3].replace('_', ' ')
                elem[2] = elem[2].replace('_', ' ')
                lista_dia.append(f'{elem[3]} \n {elem[2]}')

    lista_auxiliar.append(lista_dia)
    lista_auxiliar = lista_auxiliar[1:]

    len_maxima = 0
    for dia in lista_auxiliar:
        if len(dia) > len_maxima:
            len_maxima = len(dia)

    horario.add_column('', [i+1 for i in range(len_maxima - 1)])

    for i in range(len(lista_auxiliar)):
        for j in range(len_maxima - len(lista_auxiliar[i])):
            lista_auxiliar[i].append('')
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del curso {curso}: ')   
    return horario

curso = '1A'
generar_horario(curso)

Horario del curso 1A: 


,Lunes,Martes,Miércoles,Jueves,Viernes
1,Matemáticas Pérez,Matemáticas Pérez,Lenguaje Rodríguez,Lenguaje Rodríguez,Lenguaje Rodríguez
2,Matemáticas Pérez,Lenguaje Rodríguez,Lenguaje Rodríguez,Música González,Lenguaje Rodríguez
3,Música González,Lenguaje Rodríguez,Historia Rodríguez,Orientación González,Matemáticas Pérez
4,Lenguaje Rodríguez,Historia Rodríguez,Religión González,Educación Física González,Educación Física González
5,Tecnología González,Historia Rodríguez,Matemáticas Pérez,Educación Física González,Educación Física González
6,Ciencias Naturales Rodríguez,Artes Visuales Pérez,Ciencias Naturales Rodríguez,Religión González,Ciencias Naturales Rodríguez
7,,Artes Visuales Pérez,,Matemáticas Pérez,
